![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [36]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [37]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
print(ds_jobs_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [38]:
ds_jobs_transformed.isnull().sum()


student_id                   0
city                         0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
job_change                   0
dtype: int64

In [39]:
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].fillna(ds_jobs_transformed['gender'].mode()[0])
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].fillna(ds_jobs_transformed['enrolled_university'].mode()[0])
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].fillna(ds_jobs_transformed['education_level'].mode()[0])
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].fillna(ds_jobs_transformed['major_discipline'].mode()[0])
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].fillna(ds_jobs_transformed['company_type'].mode()[0])
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].fillna(ds_jobs_transformed['last_new_job'].mode()[0])

In [40]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  19158 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     19158 non-null  object 
 6   education_level         19158 non-null  object 
 7   major_discipline        19158 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            19158 non-null  object 
 11  last_new_job            19158 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [41]:
## Columns containing categories with two factors to be stored as (bool)
for col in ds_jobs_transformed.columns:
    if ds_jobs_transformed[col].nunique() == 2:
        unique_values = ds_jobs_transformed[col].unique()
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map({unique_values[0]: True, 
                                                                    unique_values[1]: False})
ds_jobs_transformed.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                   bool
dtype: object

In [42]:
# Columns containing integers only to be stored as (int32)
for col in ds_jobs_transformed.columns:
    if pd.api.types.is_integer_dtype(ds_jobs_transformed[col]):
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

In [43]:
# Columns containing floats to be stored as (float16)
for col in ds_jobs_transformed.columns:
    if pd.api.types.is_float_dtype(ds_jobs_transformed[col]):
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

In [44]:
# Columns containing nominal categorical data to be stored as (category)
for col in ds_jobs_transformed.columns:
    if ds_jobs_transformed[col].dtype == 'object':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [45]:
# columnscontaining ordinal categorical data to be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the columns:
# Specifying order of ordinal columns:
ordinal_columns = {
    'enrolled_university': ['Full time course', 'part time course', 'no_enrollment'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'major_discipline': ['STEM', 'Business Degree', 'Humanities', 'Arts', 'Other', 'No Major'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['1', '2', '3', '4', '>4', 'never']
}
# looping the categories accordingly
for col, order in ordinal_columns.items():
    ds_jobs_transformed[col] = pd.Categorical(
    ds_jobs_transformed[col],
    categories = order,
    ordered = True)
    
ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [46]:
# The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

# copying dataset
ds_jobs_transformed_copy = ds_jobs_transformed.copy()

# Fillig null values with <10
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].fillna('<10')

In [47]:
# Convert the 'experience' column to an ordered categorical type
experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
ds_jobs_transformed['experience'] = pd.Categorical(ds_jobs_transformed['experience'], categories=experience_order, ordered=True)

# Convert the 'company_size' column to an ordered categorical type
company_size_order = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
ds_jobs_transformed['company_size'] = pd.Categorical(ds_jobs_transformed['company_size'], categories=company_size_order, ordered=True)

# Now apply the filter using ordered categorical comparison
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10') &
    (ds_jobs_transformed['company_size'] >= '1000-4999')
]

# Verify the filtered DataFrame
ds_jobs_transformed.head()


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,NaN,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


In [48]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  2201 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2098 non-null   category
 6   education_level         2201 non-null   category
 7   major_discipline        2201 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2201 non-null   category
 11  last_new_job            2201 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)